In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
import os
os.getcwd()  # Prints the current working directory

'c:\\programowanie\\DCFmodeling'

In [3]:
data = spark.range(0, 5)
data.write.format("delta").save("c:/programowanie/DCFmodeling/tmp/delta-table")

In [14]:
df = spark.read.format("delta").load("c:/programowanie/DCFmodeling/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  0|
|  3|
|  1|
|  2|
|  4|
+---+



In [5]:
data = spark.range(10, 15)
data.write.format("delta").save("c:/programowanie/DCFmodeling/tmp/delta-table2")

In [15]:
df2 = spark.read.format("delta").load("c:/programowanie/DCFmodeling/tmp/delta-table2")
df.show()

+---+
| id|
+---+
|  0|
|  3|
|  1|
|  2|
|  4|
+---+



In [17]:
df.createOrReplaceTempView("test")
df2.createOrReplaceTempView("test2")

In [20]:
spark.sql("""
          SELECT * FROM test
          except
          SELECT * FROM test2
          """).show()

+---+
| id|
+---+
|  0|
|  3|
|  1|
|  2|
|  4|
+---+

